In [6]:
import os
import subprocess
import xarray as xr
import numpy as np
import pandas as pd
import pyproj as proj
import netCDF4 as nc4
from flask_restful import Resource
import multiprocessing
import urllib.request
from urllib.error import URLError

from pathlib import Path, PurePath, PurePosixPath
from datetime import datetime

import sys
def printf(format, *args):
    sys.stdout.write(format % args)

In [7]:
class Model(Resource):
    def __init__(self):
        self.metadata = {}
        self.parameters = {}
        self.outputs = {}
        self.tolerance = 0

    def get(self):
        return jsonify({
            "metadata"  : self.metadata,
            "parameters": self.parameters,
            "outputs"   : self.outputs
        })

    def get_metadata(self):
        return self.metadata

    def get_parameters(self):
        return self.parameters

    def get_outputs(self):
        return self.outputs

    def get_tolerance(self):
        return self.tolerance

    pass

In [8]:
class DeadFuelModel(Model):
    def __init__(self):

        # Prefixes
        vapour_prefix = 'VP3pm'
        temp_prefix = 'Tmx'
        precipitation_prefix = 'P'
        dead_fuel_moisture_prefix = 'DFMC'

        path = os.path.abspath('/media/arawlins/Data/UOM/lfmc-pipeline/NolanDead')+'/'

        vapour_url = "http://www.bom.gov.au/web03/ncc/www/awap/vprp/vprph15/daily/grid/0.05/history/nat/"
        max_avg_temp_url = "http://www.bom.gov.au/web03/ncc/www/awap/temperature/maxave/daily/grid/0.05/history/nat/"
        precipitation_url = "http://www.bom.gov.au/web03/ncc/www/awap/rainfall/totals/daily/grid/0.05/history/nat/"
        vapour_path = path + vapour_prefix + "/"
        max_avg_temp_path = path + temp_prefix + "/"
        precipitation_path = path + precipitation_prefix + "/"

        self.tolerance = 0.06  # As a percentage accuracy

        self.metadata = {
            "rainfall"            : "Expressed as percentage gain in water saturation",
            "spatial_resolution"  : "0.05 degrees",
            "data_x_resolution"   : 886,
            "data_y_resolution"   : 691,
            "temporal_granularity": "24 hours",
            "tolerance"           : "+/- 6%"
        }

        self.parameters = {
            "vapour pressure"            : {
                "var"               : "VP3pm",
                "path"              : vapour_path,
                "url"               : vapour_url,
                "prefix"            : vapour_prefix,
                "suffix"            : ".grid",
                "compression_suffix": ".Z"
            },
            "maximum average temperature": {
                "var"               : "T",
                "path"              : max_avg_temp_path,
                "url"               : max_avg_temp_url,
                "prefix"            : temp_prefix,
                "suffix"            : ".grid",
                "compression_suffix": ".Z"
            },
            "precipitation"              : {
                "var"               : "P",
                "path"              : precipitation_path,
                "url"               : precipitation_url,
                "prefix"            : precipitation_prefix,
                "suffix"            : ".grid",
                "compression_suffix": ".Z"
            }
        }

        self.outputs = {
            "fuel moisture": {
                "path"              : path + dead_fuel_moisture_prefix + "/",
                "url"               : "",
                "prefix"            : dead_fuel_moisture_prefix,
                "suffix"            : ".grid",
                "compression_suffix": ".Z"
            }
        }
        
    def get(self):
        return jsonify({"metadata": self.metadata, "parameters": self.parameters})
    
    @staticmethod
    def calculate(vp, t, p):
        ea = vp * 0.1
        es = 0.6108 * np.exp(17.27 * t / (t + 237.3))
        d = np.clip(ea - es, None, 0)
        return 6.79 + (27.43 * np.exp(1.05 * d))


In [9]:


def yearslist(f, l):
    """ Just a list of years in the form YYYY """
    yl = []
    for r in range(f.year, l.year):
        yl.append(r)
    return yl

def datelist(year):
    
    # eg., pd.date_range('2000-01-01', periods=365) won't work because BOM has no data after 26/12 each year
    
    start = datetime(year, 1, 1)
    end = datetime(year, 12, 26)  # BOM Christmas shutdown?? Not sure why happens in every dataset!
    return pd.to_datetime(pd.date_range(start, end, freq='D').tolist())

In [10]:
# def fast_collect(model, years):
#     """ Assign a thread per year to speed up collection process. """
#     pool = multiprocessing.Pool(processes=8)
#     pool.map(collect, model, years)


def collect(model, year):
    """ Collects all input parameters for the model as determined by the metadata. """
    for which in model.parameters:
        
        file_path = Path(model.parameters[which]['path'])
        if not file_path.is_dir():
            os.makedirs(file_path)
        
        
        for when in datelist(year):
            delta = datetime.now() - when
            
            if delta.days > 0:
                data_file = file_path / (model.parameters[which]['prefix'] + "_" +\
                        when.strftime("%Y%m%d") +\
                        model.parameters[which]['suffix'])
            
                archive_file = Path(str(data_file) + model.parameters[which]['compression_suffix'])
            
                if not data_file.is_file():
                    print("> Can't find local file for: ", when.strftime('%d/%m/%Y'))
                
                    if not archive_file.is_file():
                        print(">> Downloading...")
                        res = when.strftime("%Y%m%d") +\
                            when.strftime("%Y%m%d") +\
                            model.parameters[which]['suffix'] +\
                            model.parameters[which]['compression_suffix']
                        url = model.parameters[which]['url']
                        uri = url + res
                        printf(">> Using: %s \n>> to retrieve: %s \n>> Saving to: %s", url, res, archive_file)
                        try:
                            urllib.request.urlretrieve(uri, archive_file)
                        except URLError as e:
                            if hasattr(e, 'reason'):
                                print('We failed to reach a server.')
                                print('Reason: ', e.reason)
                            elif hasattr(e, 'code'):
                                print('The server couldn\'t fulfill the request.')
                                print('Error code: ', e.code)
                        print('>>> Download complete.')
                    
                    if archive_file.is_file():
                        print("> Have: ", when.strftime('%d/%m/%Y'))
                        print(">> Expanding: " + str(archive_file))
                        subprocess.run(["uncompress", "-k", archive_file, "/dev/null"], stdout=subprocess.PIPE)

def consume(which, when):
    """ Loads the data from the file for the given parameter. """
    fpath = model.parameters[which]['path'] +\
                model.parameters[which]['prefix'] + "_" +\
                when.strftime("%Y%m%d") +\
                model.parameters[which]['suffix']
    
    rio = rasterio.open(fpath, 'r')
    data = rio.read(1)
    rio.close()
    
    return data

def store(data, out, when):
    """ Writes the data to an ArcGrid file """
    # DFMC_meta = VP3pm_tmp.meta.copy()
    profile = {}

    # ArcGrid
    with rasterio.open(FileNameWrangler.absolute_filepath(self, out, when), 'w', **profile) as dest:
        dest.write(data, 1)
        dest.close()

In [11]:
m = DeadFuelModel()
yl = [2017, 2016, 2015, 2014, 2013, 2012]

for p in m.get_parameters():
    print(m.parameters[p]['prefix'])

VP3pm
Tmx
P


In [12]:
for y in yl:
    print(y)
    print(m.parameters)
    collect(m, y)
    
# fast_collect(m, yl)

2017
{'vapour pressure': {'var': 'VP3pm', 'path': '/media/arawlins/Data/UOM/lfmc-pipeline/NolanDead/VP3pm/', 'url': 'http://www.bom.gov.au/web03/ncc/www/awap/vprp/vprph15/daily/grid/0.05/history/nat/', 'prefix': 'VP3pm', 'suffix': '.grid', 'compression_suffix': '.Z'}, 'maximum average temperature': {'var': 'T', 'path': '/media/arawlins/Data/UOM/lfmc-pipeline/NolanDead/Tmx/', 'url': 'http://www.bom.gov.au/web03/ncc/www/awap/temperature/maxave/daily/grid/0.05/history/nat/', 'prefix': 'Tmx', 'suffix': '.grid', 'compression_suffix': '.Z'}, 'precipitation': {'var': 'P', 'path': '/media/arawlins/Data/UOM/lfmc-pipeline/NolanDead/P/', 'url': 'http://www.bom.gov.au/web03/ncc/www/awap/rainfall/totals/daily/grid/0.05/history/nat/', 'prefix': 'P', 'suffix': '.grid', 'compression_suffix': '.Z'}}
> Can't find local file for:  05/12/2017
>> Downloading...
>> Using: http://www.bom.gov.au/web03/ncc/www/awap/vprp/vprph15/daily/grid/0.05/history/nat/ 
>> to retrieve: 2017120520171205.grid.Z 
>> Saving to

In [13]:
def constructFilename(prefix, suffix, start):
    return prefix+"_"+start.strftime("%Y%m%d")+suffix

def DFMC_Calculate(date):

    DFMC_filename = constructFilename(DFMC_prefix,'.grid',date)
    P_filename = constructFilename(precip_prefix, '.grid', date)

    # load the file using rasterio
    VP3pm_filename = constructFilename(vapour_prefix, suffix, date)
    VP3pm_tmp = rasterio.open(vapour_path+VP3pm_filename, 'r')
    VP3pm_im = VP3pm_tmp.read(1)

    # Normalization...
    #VP3pm_norm = plt.Normalize(vmin=VP3pm_im.min(), vmax=VP3pm_im.max())
    #VP3pm_image = cmap(VP3pm_norm(VP3pm_im))

    # OPTION - Save color plots for later use...
    #plot = vapour_path+VP3pm_filename+'.png'

    #if not os.path.isfile(plot):
    #    plt.title(VP3pm_filename)
    #    plt.imsave(vapour_path+VP3pm_filename+'.png', VP3pm_im)
    #    plt.imshow(VP3pm_image, cmap)
    #    plt.show()

    # in hPa from AWAP, so multiply by 0.1 to convert to KPa
    ea_tmp2 = VP3pm_im*0.1


    Tmx_filename = constructFilename(temp_prefix, '.grid', date)
    Tmx_tmp = rasterio.open(max_avg_temp_path+Tmx_filename, 'r')
    Tmx_im = Tmx_tmp.read(1)

    # Normalization...
    #Tmx_norm = plt.Normalize(vmin=Tmx_im.min(), vmax=Tmx_im.max())
    #Tmx_image = cmap(Tmx_norm(Tmx_im))

    # OPTION - Save color plots for later use...
    #plt.imsave(max_avg_temp_path+Tmx_filename+'.png', Tmx_im)
    #plt.title(max_avg_temp_path + Tmx_filename)
    #plt.imshow(Tmx_image, cmap)
    #plt.show()

    # From daily mean temperature (T) we can compute saturation vapour pressure:
    es_tmp = 0.6108*np.exp(17.27*Tmx_im/(Tmx_im + 237.3)) # In KPa!!

    # ...and VPD in KPa, constrain to 0 when calculated RH >100%
    # Switch commented lines to activate clipping/clamping

    # CLAMPING OFF!!!!
    D_tmp = np.clip(ea_tmp2 - es_tmp, None, 0)
#        D_tmp = np.clip(ea_tmp2 - es_tmp, 0, None)
#        D_tmp = ea_tmp2 - es_tmp

    # compute DFMC (%)
    # DFMC model (Resco et al., AFM 2016) with the calibration coefficients for SE Australia from Nolan et al. 2016. (RSE)
    # DFMCfun <- function(x) {
    #  y <- 6.79+27.43*exp(1.05*x)
    #  return(y)}
    DFMC_tmp = 6.79+(27.43*np.exp(1.05*D_tmp))
    #DFMC_summary = pd.DataFrame(DFMC_tmp)
    #print(DFMC_summary.describe())

    # save the image

    #plt.title(DFMC_PATH+DFMC_filename)
    #DFMC_norm = plt.Normalize(vmin=DFMC_tmp.min(), vmax=DFMC_tmp.max())
    #DFMC_image = cmap(DFMC_norm(DFMC_tmp))
    DFMC_image = viridis(DFMC_tmp)

    #print(DFMC_image.shape)

    DFMC_meta = VP3pm_tmp.meta.copy()

    print(DFMC_meta)

    # ArcGrid
    with rasterio.open(DFMC_PATH+DFMC_filename, 'w', **DFMC_meta) as dest:
        dest.write(DFMC_tmp, 1)
        dest.close()

    # PNG
    plt.imsave(DFMC_PATH+DFMC_filename+'.8bit.tif', DFMC_tmp, cmap=plt.cm.viridis_r)

    plt.imshow(DFMC_tmp, cmap=plt.cm.viridis_r)
    plt.show()


    P_tmp = rasterio.open(precipitation_path+P_filename, 'r')
    P_im = P_tmp.read(1)


    #P_norm = plt.Normalize(vmin=P_im.min(), vmax=P_im.max())
    #P_image = cmap(P_norm(P_im))
    #plt.imsave(precipitation_path+P_filename+'.tif', P_im)
    return DFMC_tmp